In [ ]:
from csae_pyutils import gsheet_to_df
from tqdm import tqdm
import pandas as pd
from normdata.utils import import_from_normdata

In [ ]:
df = gsheet_to_df("1Txis3KRl9iO2Ft3Y0HbZL7WZ6ZrBQrd2jwfDNAqgQKA")

In [ ]:
col, _ = Collection.objects.get_or_create(name="Der Brenner")
domain = "brenner"
col_type, _ = CollectionType.objects.get_or_create(name="Projekt")
col.description = "https://brenner.oeaw.ac.at/"
col.collection_type = col_type
col.save()

In [ ]:
broken_gnd = []
pmb_uris = []
for i, row in tqdm(df.head(20).iterrows()):
    if isinstance(row["gnd"], str):
        gnd_uri = row["gnd"].replace("https://reconcile.gnd.network/gnd", "https://d-nb.info/gnd")
        domain_uri = f"https://brenner.oeaw.ac.at/?p=brenner{row['code']}".replace("_", "")
        entity = import_from_normdata(gnd_uri, "person")
        if entity:
            pmb_uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
            pmb_uri.entity = entity
            pmb_uri.save()
            entity.collection.add(col)
            pmb_uris.append([row['code'], entity.id])
        else:
            broken_gnd.append(gnd_uri)


In [ ]:
df = pd.DataFrame(pmb_uris)
df.to_csv('brenner_pmb.csv', index=False)

In [ ]:
df = pd.DataFrame(broken_gnd)
df.to_csv('brenner_broken_gnd.csv', index=False)